In [14]:
import bs4 
import requests
import pandas as pd
BASE_URL = "https://www.cityofboston.gov/assessing/search/"

In [31]:
data_folder = "data/2019"

file_name = "geocoded_enriched_merged_sales.csv"

data = pd.read_csv(f"{data_folder}/{file_name}")

# print rows that have None in any of the columns here
#  "total_room_num": None,
#             "bedrooms": None,
#             "bathrooms": None,
#             "half_bathrooms": None,
#             "kitchens": None,
#             "fireplaces": None,
#             "ac_type": None,
#             "heat_type": None,
#             "interior_condition": None,
#             "parking_spots": None,
#             "year_built": None,
#             "exterior_condition": None,
#             "foundation": None,
#             "full_address": None,
#             "owner_name": None,
#             "owner_property_count": None
noted_cols = ["total_room_num",\
            "bedrooms",\
            "bathrooms",\
            # "half_bathrooms",\
            "kitchens",\
            # "fireplaces",\
            # "ac_type",\
            # "heat_type",\
            "interior_condition",\
            "parking_spots",\
            "year_built",\
            "exterior_condition",\
            # "foundation",\
            "full_address",\
            "owner_name",\
            "owner_property_count"]

# get rows that have None in any of the columns
# data[noted_cols].isna().any(axis=1)

noted_cols.append("parcel")
# print rows that have None in any of the columns
# print the dimension of the dataframe
print(data[data[noted_cols].isna().any(axis=1)].shape)

parcels_that_have_none = data[data[noted_cols].isna().any(axis=1)]["parcel"]
data[data[noted_cols].isna().any(axis=1)][noted_cols]





(135, 27)


,total_room_num,bedrooms,bathrooms,kitchens,interior_condition,parking_spots,year_built,exterior_condition,full_address,owner_name,owner_property_count,parcel
13,NaN,NaN,NaN,NaN,NaN,NaN,2022,NaN,38 LEXINGTON ST BOSTON MA 02128,38 LEXINGTON STREET CONDOMINIUM TRUST,1.0,01-02620-000
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-05327-000
17,NaN,NaN,NaN,NaN,NaN,NaN,2022,NaN,47 MORRIS ST BOSTON MA 02128,47 MORRIS STREET CONDOMINIUM TRUST,1.0,01-06408-000
161,NaN,NaN,NaN,NaN,NaN,NaN,2003,NaN,1400 COLUMBIA RD BOSTON MA 02127,1400 COLUMBIA ROAD CONDOMINIUM TRUST,1.0,07-01502-000
227,NaN,NaN,NaN,NaN,NaN,NaN,1900,NaN,23 FOLSOM ST BOSTON MA 02125,23 FOLSOM STREET CONDOMINIUM TRUST,1.0,13-00147-000
...,...,...,...,...,...,...,...,...,...,...,...,...
5891,NaN,NaN,NaN,NaN,NaN,NaN,1900,NaN,67 FLORENCE ST BOSTON MA 02131,67 FLORENCE STREET CONDOMINIUM TRUST,1.0,19-03520-000
5893,NaN,NaN,NaN,NaN,NaN,NaN,1925,NaN,14 HATHAWAY ST BOSTON MA 02130,14 HATHAWAY STREET CONDOMINIUM TRUST,1.0,19-01521-000
5913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20-02618-000
5919,NaN,NaN,NaN,NaN,NaN,NaN,1890,NaN,11 PRIMROSE ST BOSTON MA 02131,11 PRIMROSE STREET CONDOMINIUM TRUST,1.0,20-05348-000


In [32]:
print(parcels_that_have_none)

13      01-02620-000
15      01-05327-000
17      01-06408-000
161     07-01502-000
227     13-00147-000
            ...     
5891    19-03520-000
5893    19-01521-000
5913    20-02618-000
5919    20-05348-000
5941    21-00800-000
Name: parcel, Length: 135, dtype: object


In [33]:
# update the parcels_that_have_none with correct info

from crawl_sales_info import get_sales_info, process_basic_info

new_df = data.copy()

# update the dataframe 
for parcel in parcels_that_have_none:
    soup = get_sales_info(parcel)
    info = process_basic_info(soup, False)
    # update info
    noted_cols_without_parcel = noted_cols[:-1]
    print(parcel, info)
    # new_df.loc[new_df["parcel"] == parcel, noted_cols_without_parcel] = info
    # update the dataframe
    for col in info.keys():
        new_df.loc[new_df["parcel"] == parcel, col] = info[col]

# save the new dataframe

new_df.to_csv(f"{data_folder}/{file_name}", index=False)



https://www.cityofboston.gov/assessing/search/?pid=01-02620-000
01-02620-000 {'total_room_num': None, 'bedrooms': None, 'bathrooms': None, 'half_bathrooms': None, 'kitchens': None, 'fireplaces': None, 'ac_type': None, 'heat_type': None, 'interior_condition': None, 'parking_spots': None, 'year_built': '2022', 'exterior_condition': None, 'foundation': None, 'full_address': '38  LEXINGTON ST BOSTON MA 02128', 'owner_name': '38 LEXINGTON STREET CONDOMINIUM TRUST', 'owner_property_count': None, 'properties': None}
https://www.cityofboston.gov/assessing/search/?pid=01-05327-000
01-05327-000 {'total_room_num': None, 'bedrooms': None, 'bathrooms': None, 'half_bathrooms': None, 'kitchens': None, 'fireplaces': None, 'ac_type': None, 'heat_type': None, 'interior_condition': None, 'parking_spots': None, 'year_built': None, 'exterior_condition': None, 'foundation': None, 'full_address': None, 'owner_name': None, 'owner_property_count': None, 'properties': None}
https://www.cityofboston.gov/assessin

/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_11922/755371059.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_df.loc[new_df["parcel"] == parcel, col] = info[col]
/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_11922/755371059.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_df.loc[new_df["parcel"] == parcel, col] = info[col]
/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_11922/755371059.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3' has dtype incompatible with float64, please explicitly cast to a 

16-02503-000 {'total_room_num': None, 'bedrooms': None, 'bathrooms': None, 'half_bathrooms': None, 'kitchens': None, 'fireplaces': None, 'ac_type': None, 'heat_type': None, 'interior_condition': None, 'parking_spots': None, 'year_built': '2021', 'exterior_condition': None, 'foundation': None, 'full_address': '32  NORWOOD ST BOSTON MA 02122', 'owner_name': '28 NORWOOD  LLC', 'owner_property_count': None, 'properties': None}
https://www.cityofboston.gov/assessing/search/?pid=17-02280-000
17-02280-000 {'total_room_num': None, 'bedrooms': None, 'bathrooms': None, 'half_bathrooms': None, 'kitchens': None, 'fireplaces': None, 'ac_type': None, 'heat_type': None, 'interior_condition': None, 'parking_spots': None, 'year_built': '1890', 'exterior_condition': None, 'foundation': None, 'full_address': '86  BLOOMFIELD ST BOSTON MA 02124', 'owner_name': '86 BLOOMFIELD STREET CONDOMINIUM TRUST', 'owner_property_count': None, 'properties': None}
https://www.cityofboston.gov/assessing/search/?pid=17-04

In [30]:
# check the new dataframe if there are any None in the noted_cols
print("Number of rows with missing values:", new_df[new_df[noted_cols].isna().any(axis=1)].shape[0])

# Display rows that still have missing values, if any
missing_rows = new_df[new_df[noted_cols].isna().any(axis=1)][noted_cols]
if not missing_rows.empty:
    print("\nRows with missing values:")
    print(missing_rows)
else:
    print("\nNo missing values found in the specified columns")


Number of rows with missing values: 85

Rows with missing values:
     total_room_num bedrooms bathrooms kitchens interior_condition  \
10              NaN      NaN       NaN      NaN               None   
16              NaN      NaN       NaN      NaN               None   
21              NaN      NaN       NaN      NaN               None   
162             NaN      NaN       NaN      NaN               None   
271             NaN      NaN       NaN      NaN               None   
...             ...      ...       ...      ...                ...   
6267           None     None      None     None               None   
6302           None     None      None     None               None   
6305           None     None      None     None               None   
6325           None     None      None     None               None   
6328           None     None      None     None               None   

     parking_spots year_built exterior_condition  \
10             NaN       1890            